In [1]:
import os    
#os.environ['THEANO_FLAGS'] = "device=gpu1"  
#os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32,lib.cnmem=1"
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32"
import theano
import numpy as np
from keras.layers import Input, Dense, convolutional,Reshape, Flatten
from keras.models import Model
from keras.optimizers import *

from sklearn.cross_validation import train_test_split

%matplotlib inline






Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: GeForce 920M (CNMeM is disabled, cuDNN not available)
Using Theano backend.
/home/mrafi123/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [22]:
# this is our input placeholder
input_img = Input(shape=(3,32,32))

x = convolutional.Convolution2D(16, 5, 5, activation='relu', border_mode='same')(input_img)
x = convolutional.MaxPooling2D((2, 2), border_mode='same')(x)
x = convolutional.Convolution2D(20, 5, 5, activation='relu', border_mode='same')(x)
x = convolutional.MaxPooling2D((2, 2), border_mode='same')(x)
x = convolutional.Convolution2D(20, 5, 5, activation='relu', border_mode='same')(x)
x = convolutional.MaxPooling2D((2, 2), border_mode='same')(x)
x=  Flatten()(x)
x = Dense(40, activation='relu')(x)
classifier = Dense(10, activation='softmax')(x)
model = Model(input_img, classifier)



model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])

In [4]:
#autoencoder.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy') #around 35.47% test accuracy
###->autoencoder.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy']) #around 40% test accuracy
#autoencoder.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy') #around 10% test accuracy
#autoencoder.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy') #around 10% test accuracy

In [5]:
X_train=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/X_train.npy')
X_test=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/X_test.npy')
y_train=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/y_train.npy')
y_test=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/y_test.npy')

In [6]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [11]:
from keras.utils.np_utils import *

label_train=to_categorical(y_train)
label_test= to_categorical(y_test)
print (label_train.shape)
print (label_test.shape)


(50000, 10)
(10000, 10)


In [7]:
x_train = X_train.astype('float32') / 255.
x_test = X_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print (x_train.shape)
print (x_test.shape)

(50000, 3072)
(10000, 3072)


In [23]:
x_train = x_train.reshape((len(x_train),3,32,32))
x_test = x_test.reshape((len(x_test),3,32,32))
print (x_train.shape)
print (x_test.shape)

(50000, 3, 32, 32)
(10000, 3, 32, 32)


In [24]:
print (model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 3, 32, 32)     0                                            
____________________________________________________________________________________________________
convolution2d_16 (Convolution2D) (None, 16, 32, 32)    1216        input_6[0][0]                    
____________________________________________________________________________________________________
maxpooling2d_16 (MaxPooling2D)   (None, 16, 16, 16)    0           convolution2d_16[0][0]           
____________________________________________________________________________________________________
convolution2d_17 (Convolution2D) (None, 20, 16, 16)    8020        maxpooling2d_16[0][0]            
___________________________________________________________________________________________

In [25]:
model.fit(x_train, label_train,
            nb_epoch=2,
            batch_size=256,
            shuffle=True,
            validation_data=(x_test, label_test)
            )

Train on 50000 samples, validate on 10000 samples
Epoch 1/2
50000/50000 [==============================] - 67s - loss: 2.2019 - acc: 0.1987 - val_loss: 2.0791 - val_acc: 0.2439
Epoch 2/2
50000/50000 [==============================] - 67s - loss: 2.0213 - acc: 0.2680 - val_loss: 1.9762 - val_acc: 0.2749


In [64]:
y_pred = model.predict(x_test)
print (y_pred.size)
print (y_pred.shape)
print (y_pred.nbytes)

100000
(10000, 10)
800000


In [65]:
len(y_pred)

10000

In [66]:
y_pred[0]

array([ 0.08579691,  0.07687199,  0.12446834,  0.18162394,  0.10637974,
        0.12461416,  0.10059114,  0.09089096,  0.06253739,  0.04622545])

In [67]:
new_y_pred=[]
for i in range (len(y_pred)):
    new_y_pred.append([np.argmax(y_pred[i])])


In [68]:
new_y_pred=np.asarray(new_y_pred)

In [69]:
print (new_y_pred.shape)
new_y_pred=to_categorical(new_y_pred)
print (new_y_pred.shape)


(10000, 1)
(10000, 10)


In [70]:
new_y_pred

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [28]:
label_test

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.]])

In [71]:
num=len(y_pred)
r=0
w=0
for i in range(num):
        #print ('y_pred ',y_pred[i])
        #print ('labels ',labels[i])
        #without the use of all() returns error truth value of an array with more than one element is ambiguous
        #if y_pred[i].all() == labels[i].all():
        if np.array_equal(new_y_pred[i],label_test[i]):
            r+=1
        else:
            w+=1
print ("tested ",  num, "digits")
print ("correct: ", r, "wrong: ", w, "error rate: ", float(w)*100/(r+w), "%")
print ("got correctly ", float(r)*100/(r+w), "%")


tested  10000 digits
correct:  2749 wrong:  7251 error rate:  72.51 %
got correctly  27.49 %
